In [ ]:
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
POP_SIZE = 500
DOFS_IN_POP = (POP_SIZE, 2)
N_MATING = 300
N_OFFSPRING = 300
IDX_CROSSOVER = 1

curr_population = np.random.uniform(-100,100,DOFS_IN_POP)

In [ ]:
def calc_fitness(t_pop):
    sqrt3over2 = 3**0.5/2
    fitness = []
    for candidate in t_pop:
        x1 = candidate[0]
        x2 = candidate[1]
        indv_fitness = -((sqrt3over2*(x1-3) + 0.5*(x2-5))**2 + 5*(sqrt3over2*(x2-5) - 0.5*(x1-3))**2)
        fitness.append(indv_fitness)
    return np.array(fitness)

In [ ]:
def select_determinstic(t_pop, t_fitness):
    idx = np.argsort(t_fitness)
    idx = idx[::-1]
    parents = t_pop[idx]
    parents = parents[:N_MATING, :]
    return (parents, calc_fitness(parents))

In [ ]:
def select_stochastic(t_pop, t_fitness):
    idx = np.argsort(t_fitness)
    r_i = np.empty_like(idx)
    r_i[idx] = np.arange(len(t_fitness))
    p_i = r_i + 2
    sum_pi = np.sum(p_i)
    q_i = p_i / sum_pi
    q_idx = np.argsort(q_i)
    q_i = q_i[q_idx]
    q_i = q_i[::-1]
    random_increment = 1./N_MATING
    random_val = np.random.rand(1, )
    arrow_locations = random_val + random_increment * np.arange(N_MATING)
    arrow_locations %= 1
    arrow_locations.sort()
    cum_qi = np.cumsum(q_i)
    zone_belong = np.searchsorted(cum_qi, arrow_locations)
    zone_belong = POP_SIZE - zone_belong - 1
    parents = t_pop[q_idx][zone_belong]
    par_fitness = t_fitness[q_idx][zone_belong]
    return (parents, par_fitness)

In [ ]:
def crossover(t_parents):
    offspring = np.empty((N_OFFSPRING, 2))
    for k in range(N_OFFSPRING):
        p1_idx = k%N_MATING
        p2_idx = (k+1)%N_MATING
        offspring[k, :IDX_CROSSOVER] = t_parents[p1_idx, :IDX_CROSSOVER]
        offspring[k, IDX_CROSSOVER:] = t_parents[p2_idx, IDX_CROSSOVER:]
    return (offspring, calc_fitness(offspring))

In [ ]:
from lib2to3.pgen2.token import N_TOKENS
PM = 0.5
def mutation(t_offspring):
    random_mutator = np.random.uniform(0.0, 1.0, (N_OFFSPRING,))
    idx = random_mutator > PM
    nnz = np.count_nonzero(idx)
    mutated_offspring = t_offspring.copy()
    mutated_offspring[idx] = np.random.uniform(-100,100, 2)
    return (mutated_offspring, calc_fitness(mutated_offspring))

In [ ]:
def hard_constraint(t_total_pop):
    return t_total_pop

In [ ]:
def environmental_selection(t_total_pop):
    tot_fitness = calc_fitness(t_total_pop)
    idx = np.argsort(tot_fitness)
    idx = idx[::-1]
    idx = idx[:POP_SIZE]
    return t_total_pop[idx]

In [ ]:
best_outputs = []
num_generations = 1000
curr_population = np.random.uniform(-100,100,DOFS_IN_POP)
overall_max_fitness = -99999

for generation in range(num_generations):
    print("Generation : ", generation)

    fitness = calc_fitness(curr_population)

    max_fitness = np.max(fitness)

    print("Best result in current iteration {0} compared to overall {1}".format(max_fitness, max(max_fitness, overall_max_fitness)))
    best_outputs.append(max_fitness)
    
    parents, _ = select_determinstic(curr_population, fitness)

    offspring_crossed, _ = crossover(parents)

    offspring_mutated, _ = mutation(offspring_crossed)

    total_population = np.vstack((curr_population, offspring_mutated))
    total_population = hard_constraint(total_population)

    curr_population = environmental_selection(total_population)
              
fitness = calc_fitness(curr_population)

max_idx = np.argmax(fitness)

print("Best solution : ", curr_population[max_idx, :])
print("Best solution fitness : ", fitness[max_idx])

In [ ]:
import pylab as plt
import seaborn as sns
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})

%matplotlib inline
plt.figure(figsize=(12,12))
plt.plot(best_outputs,'-o', lw=3, ms=20, label='from scratch')
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.show()

In [ ]:
%matplotlib inline
plt.figure(figsize=(12,12))
plt.plot(120-np.array(best_outputs),'-o', lw=3, ms=20, label='from scratch')
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.show()